# Busca de Índice e Consulta de documento

**Autor:** Davi J. Leite Santos  
**Versão:** 0.0.3  
**Data:** 25 de Abril de 2024  
**Localização:** Ribeirão das Neves, Minas Gerais - Brasil  

## Contato
- 🏠 **Endereço:** Ribeirão das Neves, Minas Gerais - Brasil
- 📧 **Email:** davi.jls@outlook.com
- 🌐 **LinkedIn:** davi-j-leite-santos
- 🌐 **Website:** davijls.com.br

## Principais Competências
- **Cibersegurança**
- **Segurança da Informação**
- **Operações de TI**

### Reconstruir o documento
Para reconstruir o documento usando o índice invertido.

# Sobre o Codigo
Este código Python é parte de um sistema complexo projetado para criar, gerenciar e consultar um índice invertido de documentos textuais. Ele foi estruturado para realizar a indexação de arquivos de texto, o armazenamento desses índices, e operações de recuperação e conversão dos dados indexados, adequados para aplicações de busca em larga escala.

### Componentes Principais do Código

#### 1. Importações e Configurações Básicas
O código inicia com a importação de módulos essenciais como `os` para operações relacionadas ao sistema operacional e `json` para manipulação de dados em formato JSON.

#### 2. Processamento de Documentos
- **process_file:** Função que lê e processa cada arquivo de texto individual, criando um índice invertido para cada palavra encontrada. Cada palavra é convertida para minúsculas e caracteres especiais são removidos. Essa função atualiza dois dicionários: `vocab` (vocabulário) e `index` (índice invertido).
  
#### 3. Gerenciamento de Progresso
- **load_progress:** Carrega o progresso de indexação anterior a partir de um arquivo JSON.
- **save_progress:** Salva o progresso corrente da indexação em um arquivo JSON, permitindo retomar o processo posteriormente sem repetir o trabalho já realizado.

#### 4. Criação do Índice Invertido
- **create_inverted_index:** Itera sobre todos os documentos em um diretório, processando cada um e salvando os índices criados juntamente com o vocabulário em arquivos separados. Essa função garante que o processo pode ser retomado de onde parou em caso de falha.

#### 5. Fusão de Índices
- **merge_indices_and_vocabs:** Combina todos os índices e vocabulários individuais em um único arquivo geral, integrando os dados de múltiplos documentos para criar um recurso centralizado e mais fácil de gerenciar.

#### 6. Conversão e Carga de Dados
- **convert_to_list_format** e **convert_to_tuple_format:** Convertem os dados do índice geral em diferentes formatos (lista e tupla) para facilitar consultas e manipulações futuras.
- **load_geral_files** e **load_all_files2:** Carregam os arquivos gerais e formatados do índice e vocabulário para serem usados em consultas subsequentes.

#### 7. Aplicação Prática
- **correlate_vocab_with_documents:** Associa cada palavra do vocabulário aos documentos originais onde ela aparece, demonstrando a utilidade prática do índice invertido criado.

### Considerações de Design e Implementação

O código é organizado para garantir que grandes volumes de dados possam ser processados de forma eficiente. Ele usa abordagens de "fail-safe" como salvar o progresso regularmente para evitar perdas de dados em caso de falhas. Cada função é modular, permitindo reuso e manutenção fácil.

Este sistema é um exemplo de como a indexação invertida pode ser implementada e utilizada em contextos de grandes conjuntos de dados de texto, sendo uma ferramenta essencial em aplicações que requerem recuperação rápida e eficiente de informações, como motores de busca e sistemas de gerenciamento de documentos.


# ----------------------------------------------------------------

In [4]:
import os
import json


# Processo de carregar os arquivos e guarda-los

Essa parte server para acessar cada documento e fazer o indice invertido de cada um, justamente para armazena-los de alguma forma.

In [12]:
# Lendo e processando os arquivos
def process_file(file_path, file_name, vocab, index, doc_id):
    if not os.path.exists(file_path):
        print(f"File {file_path} does not exist.")
        return
    with open(file_path, "r", encoding="utf-8") as f:
        words = f.read().split()
    for pos, word in enumerate(words):
        word = word.lower().strip(".,!?;")
        if word not in vocab:
            vocab[word] = len(vocab)
        word_id = vocab[word]
        if word_id not in index:
            index[word_id] = {"doc_info": {}, "file_name": file_name}
        if doc_id not in index[word_id]["doc_info"]:
            index[word_id]["doc_info"][doc_id] = []
        index[word_id]["doc_info"][doc_id].append(pos)


# Salvando os index do indice invertido
def save_index(vocab, index, vocab_file, index_file):
    with open(vocab_file, "w", encoding="utf-8") as f:
        json.dump(vocab, f)
    with open(index_file, "w", encoding="utf-8") as f:
        json.dump(index, f)


# Lendo o progresso dos arquivos
def load_progress(progress_file):
    if os.path.exists(progress_file) and os.path.getsize(progress_file) > 0:
        with open(progress_file, "r", encoding="utf-8") as f:
            progress = json.load(f)
    else:
        progress = {"last_processed": -1}
    return progress


# Salvando o progresso de analise dos arquivos para saber onde parou caso o codigo de algum bug e pare
def save_progress(progress, progress_file):
    with open(progress_file, "w", encoding="utf-8") as f:
        json.dump(progress, f)


# Criando o indice invertido
def create_inverted_index(files_dir, index_dir, vocab_dir, progress_file):
    # Dando Load no progresso
    progress = load_progress(progress_file)

    # Para ler cada arquivo com base no progresso
    files = sorted(os.listdir(files_dir))
    for i, file_name in enumerate(files):
        if i <= progress["last_processed"]:
            continue
        file_path = os.path.join(files_dir, file_name)

        # Criando os indices
        vocab = {}
        index = {}
        process_file(file_path, file_name, vocab, index, 0)

        # Salvando em formato de arquivo
        vocab_file = os.path.join(vocab_dir, f"vocab{i+1}.json")
        index_file = os.path.join(index_dir, f"index{i+1}.json")
        save_index(vocab, index, vocab_file, index_file)

        # Salvando o progresso
        progress["last_processed"] = i
        save_progress(progress, progress_file)

        print(f"Processed {file_name}")


# Função para juntar todos indices (Vocab e Index)
def merge_indices_and_vocabs(index_dir, vocab_dir, index_geral_file, vocab_geral_file):
    geral_index = {}
    geral_vocab = {}
    current_word_id = 0

    # Organizando os arquivos
    index_files = sorted(os.listdir(index_dir))
    vocab_files = sorted(os.listdir(vocab_dir))

    # Carregando esses arquivos
    for index_file, vocab_file in zip(index_files, vocab_files):
        with open(os.path.join(index_dir, index_file), "r", encoding="utf-8") as f:
            index = json.load(f)
        with open(os.path.join(vocab_dir, vocab_file), "r", encoding="utf-8") as f:
            vocab = json.load(f)

        # Criando o indice invertido em apenas um arquivo
        for word, word_id in vocab.items():
            if word not in geral_vocab:
                geral_vocab[word] = current_word_id
                current_word_id += 1
            geral_word_id = geral_vocab[word]

            if geral_word_id not in geral_index:
                geral_index[geral_word_id] = {"doc_info": {}, "file_names": []}
            if (
                index[str(word_id)]["file_name"]
                not in geral_index[geral_word_id]["file_names"]
            ):
                geral_index[geral_word_id]["file_names"].append(
                    index[str(word_id)]["file_name"]
                )
            for doc_id, positions in index[str(word_id)]["doc_info"].items():
                doc_id = int(doc_id)
                if doc_id not in geral_index[geral_word_id]["doc_info"]:
                    geral_index[geral_word_id]["doc_info"][doc_id] = []
                geral_index[geral_word_id]["doc_info"][doc_id].extend(positions)

    # salvando esse indice maior em dois arquivos diferentes
    with open(vocab_geral_file, "w", encoding="utf-8") as f:
        json.dump(geral_vocab, f)
    with open(index_geral_file, "w", encoding="utf-8") as f:
        json.dump(geral_index, f)


In [13]:
# Defina os caminhos dos diretórios e arquivos
files_dir = "../Motor_de_busca-WebCrawler/sites_visitados"
index_dir = "indexs"
vocab_dir = "vocabs"
progress_file = "progress.json"


In [14]:
# Crie os diretórios se não existirem
os.makedirs(index_dir, exist_ok=True)
os.makedirs(vocab_dir, exist_ok=True)


In [15]:
# Crie o índice invertido
create_inverted_index(files_dir, index_dir, vocab_dir, progress_file)


Processed 500palavrasdotcom_wordpress_com.txt
Processed 500palavrasdotcom_wordpress_com].txt
Processed 500palavrasdotcom_wordpress_com]2016]10]20]ni-una-menos].txt
Processed 500palavrasdotcom_wordpress_com]2016]11]20]depressao-afeto-desafios-e-tentativas-de-compreensao].txt
Processed 500palavrasdotcom_wordpress_com]2016]11]22]o-mundo-pos-verdade].txt
Processed 500palavrasdotcom_wordpress_com]2016]11]27]fabulas-para-o-mundo].txt
Processed 500palavrasdotcom_wordpress_com]2016]11]30]a-sorrateira-disputa-entre-os-tres-poderes].txt
Processed 500palavrasdotcom_wordpress_com]2016]12]02]cuba-a-ilha-de-fidel-puebla-e-valdes].txt
Processed 500palavrasdotcom_wordpress_com]2016]12]06]vizinhos-desconhecidos-o-encontro-de-brasil-e-colombia].txt
Processed 500palavrasdotcom_wordpress_com]2016]12]11]design-thinking-uma-forma-de-pensar-diferente].txt
Processed 500palavrasdotcom_wordpress_com]2016]12]23]nao-sai-no-jornal-mas-explode-no-palco-karen-ramos-e-o-movimento-do-movimento].txt
Processed 500palavr

In [16]:
# Defina os caminhos dos arquivos gerais
index_geral_file = "index_geral.json"
vocab_geral_file = "vocab_geral.json"


In [17]:
# Mesclar os índices e vocabulários individuais nos arquivos gerais
merge_indices_and_vocabs(index_dir, vocab_dir, index_geral_file, vocab_geral_file)


# Fazendo as leituras das operações

In [18]:
def convert_to_list_format(index_geral_file, list_geral_file):
    with open(index_geral_file, "r", encoding="utf-8") as f:
        index = json.load(f)

    list_format = []
    for word_id, data in index.items():
        file_names = data["file_names"]
        for doc_id, positions in data["doc_info"].items():
            list_format.append(
                {
                    "word_id": int(word_id),
                    "doc_id": doc_id,
                    "positions": positions,
                    "file_names": file_names,
                }
            )

    with open(list_geral_file, "w", encoding="utf-8") as f:
        json.dump(list_format, f)


In [19]:
def load_geral_files(index_geral_file, vocab_geral_file, list_geral_file):
    with open(index_geral_file, "r", encoding="utf-8") as f:
        index_geral = json.load(f)
    with open(vocab_geral_file, "r", encoding="utf-8") as f:
        vocab_geral = json.load(f)
    with open(list_geral_file, "r", encoding="utf-8") as f:
        list_geral = json.load(f)

    return index_geral, vocab_geral, list_geral


In [20]:
def convert_to_tuple_format(index_geral_file, tuple_format_file):
    with open(index_geral_file, "r", encoding="utf-8") as f:
        index = json.load(f)

    tuple_format = []
    for word_id, data in index.items():
        file_names = data["file_names"]
        for doc_id, positions in data["doc_info"].items():
            tuple_format.append((int(word_id), int(doc_id), positions, file_names))

    with open(tuple_format_file, "w", encoding="utf-8") as f:
        json.dump(tuple_format, f)


In [21]:
def load_all_files2(index_geral_file, vocab_geral_file, tuple_format_file):
    with open(index_geral_file, "r", encoding="utf-8") as f:
        index_geral = json.load(f)
    with open(vocab_geral_file, "r", encoding="utf-8") as f:
        vocab_geral = json.load(f)
    with open(tuple_format_file, "r", encoding="utf-8") as f:
        tuple_format = json.load(f)

    return index_geral, vocab_geral, tuple_format


In [22]:
# Defina o caminho do arquivo de lista geral
list_geral_file = "list_geral.json"
# Converta o índice geral para o formato de lista
convert_to_list_format(index_geral_file, list_geral_file)
# Carregar os arquivos gerais
index_geral, vocab_geral, list_geral = load_geral_files(
    index_geral_file, vocab_geral_file, list_geral_file
)

# Defina o caminho do arquivo de formato de tupla
tuple_format_file = "tuple_format.json"
# Converta o índice geral para o formato de tupla
convert_to_tuple_format(index_geral_file, tuple_format_file)
# Carregar os arquivos gerais e o arquivo de formato de tupla
index_geral, vocab_geral, tuple_format = load_all_files2(
    index_geral_file, vocab_geral_file, tuple_format_file
)


# Correlação de arquivos

In [23]:
def correlate_vocab_with_documents(
    vocab_geral_file, index_geral_file, original_docs_dir
):
    # Carregar vocab e índice geral
    with open(vocab_geral_file, "r", encoding="utf-8") as f:
        vocab_geral = json.load(f)
    with open(index_geral_file, "r", encoding="utf-8") as f:
        index_geral = json.load(f)

    # Cria um dicionário para armazenar a correlação
    correlation = {}

    # Itera sobre o vocabulário e o índice
    for word, word_id in vocab_geral.items():
        if str(word_id) in index_geral:
            file_names = index_geral[str(word_id)]["file_names"]
            correlation[word] = []
            for file_name in file_names:
                file_path = os.path.join(original_docs_dir, file_name)
                if os.path.exists(file_path):
                    correlation[word].append(file_path)
                else:
                    print(f"File {file_path} not found.")

    return correlation


In [24]:
# Definindo os caminhos dos arquivos e diretórios
vocab_geral_file = "vocab_geral.json"
index_geral_file = "index_geral.json"
original_docs_dir = "../Motor_de_busca-WebCrawler/sites_visitados"


In [25]:
# Chamando a função para correlacionar vocab com os documentos originais
correlation = correlate_vocab_with_documents(
    vocab_geral_file, index_geral_file, original_docs_dir
)


In [ ]:
# Exibindo a correlação
for word, files in correlation.items():
    print(f"Word: {word}")
    for file in files:
        print(f"  Document: {file}")
